# Find proteins outside the 8q gain event (trans) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,lscc_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602,0.077350,0.175590,0.693346,0.827351,0.983326,0.921999,-0.604953,-0.363979,-0.075722,0.147324,0.018228,-0.096252
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2,NP_000005,0.986803,0.567309,0.828859,0.897858,0.848498,0.987671,-0.010696,-0.153202,-0.073016,-0.139115,0.181837,0.022644
A2ML1,NaN,NP_653271.2|NP_001269353.1,NP_653271.2|NP_001269353.1,NP_653271,0.145335,NaN,0.639139,0.656481,NaN,0.970183,1.331073,NaN,0.241978,1.267434,NaN,-0.083533
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.234511,0.606492,0.878184,0.804218,0.407805,0.888646,0.182912,0.089587,0.020537,-0.075273,0.208068,0.045529
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.157348,0.908989,0.743491,0.935735,0.724478,0.933755,-0.601837,0.022990,0.088849,-0.092006,0.326426,-0.068586
AADAT,NP_001273611.1|NP_001273612.1,NaN,NP_057312.1|NP_001273611.1,NaN,0.652043,NaN,NaN,NaN,0.731504,NaN,-0.420807,NaN,-0.041260,NaN,0.560279,NaN
AAED1,NP_714542.1,NP_714542.1,NP_714542.1,NaN,0.694771,NaN,0.754292,0.887110,0.979300,NaN,-0.196052,NaN,0.136753,-0.117229,0.034105,NaN
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258815.1,NP_078942,0.086879,0.317551,0.877713,0.869113,0.979057,0.899510,-0.409365,-0.144167,0.034488,-0.091437,-0.020575,-0.064130
AAK1,NaN,NP_055726.3,NP_055726.3,NP_055726,0.197906,0.206503,0.439421,0.448528,NaN,0.787477,-0.215584,-0.110944,-0.078541,-0.261571,NaN,-0.111739


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.077350,-0.604953
1,brca,A2M,NP_000005.2,0.986803,-0.010696
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.145335,1.331073
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.234511,0.182912
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.157348,-0.601837
5,brca,AADAT,NP_057312.1|NP_001273611.1,0.652043,-0.420807
6,brca,AAED1,NP_714542.1,0.694771,-0.196052
7,brca,AAGAB,NP_078942.3|NP_001258815.1,0.086879,-0.409365
8,brca,AAK1,NP_055726.3,0.197906,-0.215584
9,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.036811,-0.917606


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title="Chr 8q trans effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
ARFGEF1,"[brca, colon, hnscc, luad]",4
HGH1,"[brca, colon, hnscc, luad]",4
STAU2,"[brca, colon, hnscc, luad]",4
YTHDF3,"[brca, colon, hnscc, luad]",4
MRPL15,"[brca, colon, hnscc]",3
MRPL39,"[brca, colon, hnscc]",3
MRPL45,"[brca, colon, hnscc]",3
AGO2,"[brca, colon, luad]",3
CYC1,"[brca, colon, luad]",3


In [8]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

4       4
3      12
2      64
1    1426
Name: num_cancers, dtype: int64

## Save results

In [9]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")